In [1]:
import pandas as pd

In [2]:
df_thread = pd.read_csv('thread_18cff0b946f3a4d4_cleaned.csv', index_col='Unnamed: 0')
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 76 to 100
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   threadId          11 non-null     object
 1   from              11 non-null     object
 2   date              11 non-null     object
 3   body_cleaned      11 non-null     object
 4   body_cleaned_len  11 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [3]:
df_thread = df_thread.reset_index().drop('index', axis=1)
df_thread

,threadId,from,date,body_cleaned,body_cleaned_len
0,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 15:59:44-03:00,"Prezada DGBD, boa tarde!\r\n\r\nEstamos com er...",105
1,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 16:29:16-03:00,Copiando para o e-mail correto da DIBD.\r\n\r\...,15
2,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 17:09:03-03:00,Parece ser de tnsnames.ora desatualizado na es...,36
3,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:29:30-03:00,"Obrigado, Edson.",2
4,18cff0b946f3a4d4,Evandro Bassanesi <evandro.bassanesi@trt4.jus.br>,2024-01-12 17:34:50-03:00,"Felipe,\r\n\r\nEste está desatualizado. O banc...",24
5,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:46:45-03:00,Vamos atualizar. Obrigado!\r\n\r\n\r\nAtencios...,18
6,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 08:58:54-03:00,"Felipe, isso não ocorre no servidor do Markvis...",104
7,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 09:19:39-03:00,"Agora que entendi, o problema que refere, não...",46
8,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 10:18:12-03:00,A GPO de atualização do TNSnames (desktop-all-...,28
9,18cff0b946f3a4d4,Francisco Flesch <francisco.flesch@trt4.jus.br>,2024-01-15 11:05:28-03:00,Eu troquei o TNSnames na última vez que deu pr...,41


In [4]:
from langchain_openai import ChatOpenAI, OpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_json = ChatOpenAI(model="gpt-4o-mini").bind(
    response_format={"type": "json_object"}
)

In [6]:
prompt_grader_importancy_system = """
    You are a helpful assistant who is in charge of classifying strings as either interesting to a use case or not.
    This strings will be used by another llm to generate a report and a knowledge base fo a IT company.
    So the use case is to determine if the string contain helpful information to create a knowledge base or not.
    Helpefull information (grade as True): problem, information about something, solutions, questions
    Not helpfull information (grade as False): names, emails, phone numbers, someone saying that wil send email or something like this.

    You should output a json. Key shold be the key of the string, the value should be the boolean value
"""


prompt_grader_importancy_user = """
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_grader = ChatPromptTemplate.from_messages(
    [("system", prompt_grader_importancy_system), ("user", prompt_grader_importancy_user)]
)

In [59]:
# chain_grader = prompt_grader | llm
chain_grader = prompt_grader | llm_json

In [12]:
import json

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    length_function=len,
)

def split_email(row):
    splits = splitter.split_text(row)

    # strings = { k:v for (k,v) in zip(range(len(splits)), splits) }

    return splits

    # remove all uninformative strings
    # response = chain_grader.invoke( { 'STRINGS': strings } )
    # strings_graded = response.content

    # # # parse json
    # strings_graded = json.loads(strings_graded)

    # # # get useful strings
    # strings_graded_filtered = [ strings[int(k)] for (k,v) in strings_graded.items() if v == True ]

    # return strings_graded_filtered

In [20]:
for row in df_thread.apply(lambda row: split_email(row['body_cleaned']), axis=1):
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    for string in row:
        print(string)
        print('---------------------')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Prezada DGBD, boa tarde!
---------------------
Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem
---------------------
dar uma olhada, por favor?
---------------------
[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
---------------------
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
---------------------
> argument(s): "ORA-12541: TNS: nÆo h  listener"




Atenciosamente,

Felipe Giacomel
---------------------
Divisão de Microinformática
TRT4 - Secretaria de Tecnologia da Informação e Comunicações

--
---------------------
You received this message because you are subscribed to the Google Groups "SETIC-DIM- Divisao de
---------------------
Microinformatica" group.
---------------------
To unsubscribe from this group and stop receiving emails from it, send an email to
----------------